In [1]:
!pip install -U evaluate sacrebleu unbabel-comet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')
evaluation_df = pd.read_csv('/content/drive/MyDrive/evaluation_df.csv')

Mounted at /content/drive


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


model_save_path = '/content/drive/My Drive/models-finetuning-1/finetuned-nllb200_hr_nl/finetuned-nllb200-hr-to-nl-mmt/checkpoint-46095'

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained(model_save_path)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_save_path)

print('Model and tokenizer have been loaded successfully.')

Model and tokenizer have been loaded successfully.


In [4]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): Embedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): Embedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm

In [5]:
def translate_croatian_to_dutch_batch(texts):
    # Set the tokenizer's source language code for Croatian. Use 'hrv_Latn' for NLLB-200.
    tokenizer.src_lang = "hrv_Latn"
    # Encode the Croatian texts
    encoded_hr = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=64).to('cuda')
    # Generate the Dutch translations. Use 'nld_Latn' for the target language code.
    generated_tokens = model.generate(**encoded_hr, forced_bos_token_id=tokenizer.lang_code_to_id["nld_Latn"])
    # Decode the generated tokens to texts
    return [tokenizer.decode(g, skip_special_tokens=True) for g in generated_tokens]

In [6]:
from tqdm.auto import tqdm
from sacrebleu.metrics import BLEU

bleu = BLEU()

# Prepare lists for translations and BLEU scores
croatian_to_dutch_translations = []
croatian_to_dutch_bleu_scores = []

batch_size = 8  # Set the batch size for processing
for i in tqdm(range(0, len(evaluation_df), batch_size)):
    batch = evaluation_df.iloc[i:i+batch_size]

    # Translate batches from Croatian to Dutch
    translated_dutch_batch = translate_croatian_to_dutch_batch(batch['Croatian'].tolist())

    # Extend the translation list with the translated batch
    croatian_to_dutch_translations.extend(translated_dutch_batch)

    # Compute BLEU scores for the Dutch translations in batch
    for translated_dutch, original_dutch in zip(translated_dutch_batch, batch['Dutch'].tolist()):
        dutch_score = bleu.corpus_score([translated_dutch], [[original_dutch]])
        croatian_to_dutch_bleu_scores.append(dutch_score.score)

# Update DataFrame with translated Dutch and BLEU scores
evaluation_df['Translated Dutch'] = croatian_to_dutch_translations
evaluation_df['Croatian to Dutch BLEU'] = croatian_to_dutch_bleu_scores

  0%|          | 0/279 [00:00<?, ?it/s]

the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [7]:
evaluation_df

,Unnamed: 0,Dutch,Croatian,Translated Dutch,Croatian to Dutch BLEU
0,0,Op maandag kondigden wetenschappers van de Sta...,U ponedjeljak su znanstvenici s Medicinskog fa...,Op maandag kondigden wetenschappers van de Sta...,32.383141
1,1,Hoofdonderzoekers zeggen dat dit kan leiden to...,Voditelji istraživanja izjavili su da bi ovo o...,De onderzoekers zeiden dat deze ontdekking het...,18.566862
2,2,De JAS 39C Gripen stortte rond 09.30 uur lokal...,JAS 39C Gripen srušio se na pistu i eksplodira...,JAS 39C Gripen crashte op het spoor en explode...,21.098000
3,3,De piloot werd geïdentificeerd als majoor Dilo...,Pilot je identificiran kao zapovjednik eskadri...,De piloot werd geïdentificeerd als de commanda...,38.677063
4,4,De lokale media meldt dat er tijdens een actie...,Lokalni mediji izvješćuju da je došlo do prevr...,Plaatselijke media rapporteren dat er een over...,4.885327
...,...,...,...,...,...
2221,212,Mijn naam is Andrea.,Moje ime je Andrea.,Mijn naam is Andrea.,100.000000
2222,213,Jupiter is een planeet.,Jupiter je planet.,Jupiter is een planeet,77.880078
2223,214,Venus is een planeet.,Venera je planet.,Venus is een planeet.,100.000000
2224,215,God is een olifant.,Bog je slon.,God is een olifant.,100.000000


In [8]:
import pandas as pd
from nltk.translate.meteor_score import single_meteor_score
from nltk import word_tokenize
import nltk
from tqdm import tqdm

# Download NLTK's tokenizer models
nltk.download('punkt')
nltk.download('wordnet')

# Initialize an empty list for the METEOR scores
croatian_to_dutch_meteor_scores = []

# Loop through each row in the DataFrame
for _, row in tqdm(evaluation_df.iterrows(), total=evaluation_df.shape[0]):
    # Tokenize the Dutch sentences
    original_dutch = word_tokenize(row['Dutch'])
    translated_dutch = word_tokenize(row['Translated Dutch'])

    # Calculate the METEOR score for Croatian to Dutch translation
    meteor_score = single_meteor_score(original_dutch, translated_dutch)
    croatian_to_dutch_meteor_scores.append(meteor_score)

# Add the METEOR scores to DataFrame
evaluation_df['Croatian to Dutch METEOR'] = croatian_to_dutch_meteor_scores

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
100%|██████████| 2226/2226 [00:04<00:00, 552.08it/s] 


In [9]:
from evaluate import load
from tqdm import tqdm

# Load the COMET metric
comet_metric = load('comet')

# Function to compute COMET scores in batches
def compute_comet_in_batches(sources, predictions, references, batch_size=100):
    scores = []
    for i in tqdm(range(0, len(sources), batch_size)):
        batch_sources = sources[i:i + batch_size]
        batch_predictions = predictions[i:i + batch_size]
        batch_references = references[i:i + batch_size]
        batch_results = comet_metric.compute(predictions=batch_predictions, references=batch_references, sources=batch_sources)
        scores.extend(batch_results["scores"])
    return scores

# Calculate the COMET scores for Croatian to Dutch
data_cro_to_dutch = {
    "sources": evaluation_df['Croatian'].tolist(),
    "predictions": evaluation_df['Translated Dutch'].tolist(),
    "references": evaluation_df['Dutch'].tolist()
}

# Calculate the COMET scores
croatian_to_dutch_comet_scores = compute_comet_in_batches(data_cro_to_dutch['sources'], data_cro_to_dutch['predictions'], data_cro_to_dutch['references'])
evaluation_df['Croatian to Dutch COMET'] = [round(score, 5) for score in croatian_to_dutch_comet_scores]

# Display the updated DataFrame
print(evaluation_df.head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:188: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
  0%|          | 0/23 [00:00<?, ?it/s]INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
  4%|▍         | 1/23 [00:04<01:40,  4.55s/it]INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0

   Unnamed: 0                                              Dutch  \
0           0  Op maandag kondigden wetenschappers van de Sta...   
1           1  Hoofdonderzoekers zeggen dat dit kan leiden to...   
2           2  De JAS 39C Gripen stortte rond 09.30 uur lokal...   
3           3  De piloot werd geïdentificeerd als majoor Dilo...   
4           4  De lokale media meldt dat er tijdens een actie...   

                                            Croatian  \
0  U ponedjeljak su znanstvenici s Medicinskog fa...   
1  Voditelji istraživanja izjavili su da bi ovo o...   
2  JAS 39C Gripen srušio se na pistu i eksplodira...   
3  Pilot je identificiran kao zapovjednik eskadri...   
4  Lokalni mediji izvješćuju da je došlo do prevr...   

                                    Translated Dutch  Croatian to Dutch BLEU  \
0  Op maandag kondigden wetenschappers van de Sta...               32.383141   
1  De onderzoekers zeiden dat deze ontdekking het...               18.566862   
2  JAS 39C Gri

In [10]:
# Calculate average scores for each metric
average_croatian_to_dutch_bleu = evaluation_df['Croatian to Dutch BLEU'].mean()
average_croatian_to_dutch_meteor = evaluation_df['Croatian to Dutch METEOR'].mean()
average_croatian_to_dutch_comet = evaluation_df['Croatian to Dutch COMET'].mean()

# Print the averages
print("Translation scores from Croatian to Dutch:")
print(f"Average BLEU: {average_croatian_to_dutch_bleu}")
print(f"Average METEOR: {average_croatian_to_dutch_meteor}")
print(f"Average COMET: {average_croatian_to_dutch_comet}")

Translation scores from Croatian to Dutch:
Average BLEU: 20.834615541300263
Average METEOR: 0.48229150401760995
Average COMET: 0.8374399460916442


In [11]:
evaluation_df.to_csv('/content/drive/MyDrive/evaluation_df_nllb200_1_hr_nl.csv')